In [1]:
import numpy as np
import pandas as pd
import time
import datetime
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

**1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).**

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')

features.shape

(97230, 108)

In [9]:
X = features.loc[:,'start_time':'dire_first_ward_time'] 
y_all = features.loc[:,'duration':'barracks_status_dire']
X.shape

(97230, 102)

X -- все признаки
y_all -- все итоговые переменные

**2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.**

In [4]:
num = X.count().to_frame()
print(num.loc[num[0] != features.shape[0]])


                                 0
first_blood_time             77677
first_blood_team             77677
first_blood_player1          77677
first_blood_player2          53243
radiant_bottle_time          81539
radiant_courier_time         96538
radiant_flying_courier_time  69751
radiant_first_ward_time      95394
dire_bottle_time             81087
dire_courier_time            96554
dire_flying_courier_time     71132
dire_first_ward_time         95404


Данные признаки(команда/игрок/время первого убийства, время первого приобретения предметов) могут иметь пропуски, т.к. соответствующие им действия могут быть не совершены за первые 5 минут матча.

**3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.**

In [5]:
X = X.fillna(0) #замена пропусков на 0

4. Какой столбец содержит целевую переменную? Запишите его название.

In [6]:
y = y_all.loc[:,'radiant_win'] #целевая переменная -- 'radiant_win'


**4. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?**

In [7]:
kf = KFold(n_splits=5, shuffle = True) #генератор разбиений для кросс-валидации по 5 блокам

In [8]:
for n in [10, 20, 30, 40, 50]:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators = n)
    score_array =[]
    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
        clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        score_array.append(roc_auc_score(y_test, pred))
    res = np.mean(score_array)
    print('При n = ' + str(n) + ' качество = ' + str(res) + ', затраченное время: ' + str(datetime.datetime.now() - start_time))

При n = 10 качество = 0.6626965769468534, затраченное время: 0:00:21.787007
При n = 20 качество = 0.6814949342550289, затраченное время: 0:00:37.653457
При n = 30 качество = 0.6898195438075352, затраченное время: 0:00:53.405873
При n = 40 качество = 0.6941188659922191, затраченное время: 0:01:17.078251
При n = 50 качество = 0.6969521163776851, затраченное время: 0:01:57.264201


# Итог

**1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?**
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
    - `radiant_(dire_)bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_(dire_)courier_time`: время приобретения предмета "courier" 
    - `radiant_(dire_)flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_(dire_)first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
    Данные признаки(команда/игрок/время первого убийства, время первого приобретения предметов) могут иметь пропуски, т.к. соответствующие им действия могут быть не совершены за первые 5 минут матча.
    
**2. Как называется столбец, содержащий целевую переменную?**
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    
**3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?**
    
    При n = 30 качество = 0.689612555952278, затраченное время: 0:00:52.914256
    
**4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?**
    
    Использование более 30 деревьев незначительно, но увеличивает значение метрики AUC-ROC. Т.е. увеличение количества деревьев имеет смысл.
    Для ускорения обучения можно попробовать уменьшить глубину деревьев, и оптимизировать количество признаков.